In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.datasets import load_boston
from sklearn.metrics import r2_score, mean_squared_log_error, mean_squared_error,mean_absolute_error
import datetime as dt
import pickle
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import preparing_data as F
import ml_module as ml


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import AdaBoostRegressor

In [2]:
data=pd.read_pickle("./dataframe/df_only_risky_events.pkl")
data.reset_index(inplace=True)
data.drop(['index'], inplace=True, axis=1)
print(data.shape)
data.head()

(272, 86)


,__time_to_tca,MISS_DISTANCE,RELATIVE_SPEED,RELATIVE_POSITION_R,RELATIVE_POSITION_T,RELATIVE_POSITION_N,RELATIVE_VELOCITY_R,RELATIVE_VELOCITY_T,RELATIVE_VELOCITY_N,COLLISSION_PROBABILITY,...,PC_mavg_1,PC_trend_1,PC_trend_3,PC_gradient_1,PC_gradient_3,MD_mavg_1,MD_trend_1,MD_trend_3,MD_gradient_1,MD_gradient_3
0,0.832257,2244.0,11172.0,87.9,-1474.9,1689.9,-201.4,-8424.3,-7336.0,-3.418392,...,-3.475175,0.132439,0.590914,0.354472,0.579354,2181.333333,47.0,17.0,125.795356,16.667421
1,0.580855,966.0,212.0,191.0,-944.8,-68.9,33.8,-8.4,209.3,-3.711974,...,-3.981494,0.310256,0.575376,0.747481,0.536846,1131.000000,-201.0,-326.0,-484.256505,-304.169574
2,0.189825,1102.0,212.0,193.3,-1083.1,-75.0,33.9,-8.4,209.3,-3.913996,...,-3.882734,-0.202022,0.296282,-0.516640,0.270366,1078.333333,136.0,-158.0,347.799165,-144.179478
3,0.994637,456.0,11707.0,-22.1,288.8,353.3,-10.2,-9077.7,7392.6,-2.055073,...,-2.841298,1.280567,1.312874,3.634346,1.445698,709.666667,-577.0,124.0,-1637.570088,136.545083
4,0.759012,629.0,11707.0,-18.7,395.3,489.9,-10.3,-9077.7,7392.6,-2.649558,...,-2.680091,-0.594485,0.483623,-2.523011,0.486607,706.000000,173.0,-11.0,734.217036,-11.067877


In [3]:
train, test = train_test_split(data, test_size=0.30, random_state=42)

In [4]:
print("Train dataframe dimension {} x {}".format(train.shape[0],train.shape[1]))
print("Test dataframe dimension {} x {}".format(test.shape[0],test.shape[1]))

Train dataframe dimension 190 x 86
Test dataframe dimension 82 x 86


In [5]:
Y_train = train["COLLISSION_PROBABILITY"]
X_train= train.drop(["COLLISSION_PROBABILITY"], axis=1)
Y_test = test["COLLISSION_PROBABILITY"]
X_test= test.drop(["COLLISSION_PROBABILITY"], axis=1)


In [6]:
X = X_train
y = Y_train

In [7]:
#model = AdaBoostRegressor()

In [8]:
# evaluate the model
# cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# n_scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
# # report performance
# print('MAE: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

In [11]:
# example of bayesian optimization with scikit-optimize
from numpy import mean
from sklearn.datasets import make_blobs
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from skopt.space import Integer, Real
from skopt.utils import use_named_args
from skopt import gp_minimize
 
# generate 2d classification dataset
# define the model
model = AdaBoostRegressor()
# define the space of hyperparameters to search
search_space = [Integer(50, 100, name='n_estimators'), Real(0.001, 0.01, name='learning_rate')]
 
# define the function used to evaluate a given configuration
@use_named_args(search_space)
def evaluate_model(**params):
	# something
	model.set_params(**params)
	# calculate 5-fold cross validation
	result = cross_val_score(model, X, y, cv=5, n_jobs=-1, scoring='r2')
	# calculate the mean of the scores
	estimate = mean(result)
	return 1.0 - estimate
 
# perform optimization
result = gp_minimize(evaluate_model, search_space)
# summarizing finding:
print('Best r2: %.3f' % (1.0 - result.fun))
print('Best Parameters: n_neighbors=%d, p=%d' % (result.x[0], result.x[1]))

Best r2: 0.815
Best Parameters: n_neighbors=98, p=0


In [14]:
result.space

Space([Integer(low=50, high=100, prior='uniform', transform='normalize'),
       Real(low=0.001, high=0.01, prior='uniform', transform='normalize')])

In [13]:
result.models

[GaussianProcessRegressor(kernel=1**2 * Matern(length_scale=[1, 1], nu=2.5) + WhiteKernel(noise_level=1),
                          n_restarts_optimizer=2, noise='gaussian',
                          normalize_y=True, random_state=700955242),
 GaussianProcessRegressor(kernel=1**2 * Matern(length_scale=[1, 1], nu=2.5) + WhiteKernel(noise_level=1),
                          n_restarts_optimizer=2, noise='gaussian',
                          normalize_y=True, random_state=700955242),
 GaussianProcessRegressor(kernel=1**2 * Matern(length_scale=[1, 1], nu=2.5) + WhiteKernel(noise_level=1),
                          n_restarts_optimizer=2, noise='gaussian',
                          normalize_y=True, random_state=700955242),
 GaussianProcessRegressor(kernel=1**2 * Matern(length_scale=[1, 1], nu=2.5) + WhiteKernel(noise_level=1),
                          n_restarts_optimizer=2, noise='gaussian',
                          normalize_y=True, random_state=700955242),
 GaussianProcessRegressor(ke